In [ ]:


#Get the path of the current working directory
import os
cwd = os.getcwd()
print(cwd)

from src.data_loader import DataLoader
from src.enumerations import *

# Instantiate the data loader

# Load the data
data_loader = DataLoader()

In [ ]:
from src.model_time import TimeHorizon
# Instantiate the TimeHorizon class
time_horizon = TimeHorizon(start_year=2015, end_year=2300, data_timestep=5, timestep=1)

# th = time_horizon.get_model_time_horizon()
#print(th)
print(time_horizon.data_time_horizon)  
# Pass the time_horizon object to other parts of the model

In [ ]:
path_to_justice = '/Users/palokbiswas/Desktop/pollockdevis_git/JUSTICE'

# import sys
# sys.path.insert(0, path_to_justice)



from src.economy.neoclassical import NeoclassicalEconomyModel
from src.emissions.emission import OutputToEmissions
#from economy.neoclassical import NeoclassicalEconomyModel

In [ ]:
model = NeoclassicalEconomyModel(input_dataset = data_loader, time_horizon=time_horizon)

In [ ]:
emissions = OutputToEmissions(input_dataset=data_loader, time_horizon=time_horizon)


In [ ]:
gdp = NeoclassicalEconomyModel.__getattribute__(model, 'gdp_dict')
#gdp = data_loader.GDP_DICT
testdf = gdp['SSP1']

population = NeoclassicalEconomyModel.__getattribute__(model, 'population_dict')
testdf2 = population['SSP1']

In [ ]:
print(model.__getattribute__('capital_elasticity_in_production_function'))

In [ ]:
print(model.__getattribute__('depreciation_rate_capital'))

In [ ]:
print(model.get_optimal_long_run_savings_rate())

In [ ]:
import numpy as np
#Create a fixed savings rate
fixed_savings_rate = np.copy(data_loader.SAVING_RATE_INIT_ARRAY).reshape(-1, 1)
#fixed_savings_rate Validated with RICE50 for timestep 1 and 5

set_year = time_horizon.model_time_horizon


for i, years in enumerate(set_year):
    
    t = i+1 #index starts at 0, so add 1 to get the year

    if t != 1: # no need to repeat for the first year

        next_rate = data_loader.SAVING_RATE_INIT_ARRAY + (model.get_optimal_long_run_savings_rate() - data_loader.SAVING_RATE_INIT_ARRAY)*((t - 1)/(len(set_year) - 1))
        # append to the fixed savings rate array for each year
        fixed_savings_rate = np.column_stack((fixed_savings_rate, next_rate))

from matplotlib import pyplot as plt
plt.plot(np.sum(fixed_savings_rate, axis=0))
plt.title("Fixed Savings Rate")
plt.xlabel("Time")
plt.ylabel("Savings Rate")
plt.show()

In [ ]:
len(set_year)

In [ ]:
scenario = SSP.SSP1
print(scenario.value)

for i in range(len(set_year)):
    output = model.run(scenario = scenario.value, timestep=i, savings_rate=fixed_savings_rate[:, i])
    #print shape of output
    print(output.shape)
    print(i)
    #should emissions be an array of shape region, timesteps?
    emis = emissions.run_emissions(timestep=i, scenario=scenario.value, output=output[:,i], emission_control_rate=0.0)


#run_emissions(self, timestep, scenario, output, emission_control_rate)

In [ ]:
print(output)

In [ ]:
import pandas as pd
testdf = pd.DataFrame(output)

## Start here for experimentation

In [1]:
from src.model_time import TimeHorizon
# Instantiate the TimeHorizon class
time_horizon = TimeHorizon(start_year=2015, end_year=2300, data_timestep=5, timestep=1)

# th = time_horizon.get_model_time_horizon()
#print(th)
# print(time_horizon.data_time_horizon)  

from src.climate.coupled_fair import CoupledFAIR

model = CoupledFAIR()

#model.fair_justice_run_init(time_horizon=time_horizon, scenarios=["ssp245"])

In [2]:
# # model.fair_fill_data(time_horizon=time_horizon, scenarios=["ssp245"])
# # model.test_init_fair(scenarios=["ssp245"], time_horizon=time_horizon)
# model.species_configs
# model.configs

In [2]:
model.fair_justice_run_init(time_horizon=time_horizon, scenarios=["ssp245"])

In [ ]:
current_emissions = model.emissions.sel(specie="CO2 FFI", scenario=["ssp245"])

#Plots the current emissions
current_emissions = current_emissions[:,0,:]

In [3]:
#create a list of years 1750 to 2014 with 1 year timestep
import time
import numpy as np

years_fair = np.arange(1750, 2300, 1) #Time period of FAIR model, it starts with historical data. from 2015, we only replace the emissions
years_justice = np.arange(2015, 2300+1, 1)  #Time period of JUSTICE model

#Loop through the number of years starting with zero for 1750
loop_counter = 0

for i in range(0, len(years_fair)):
    loop_counter = i
    
    start_time = time.time()
    model.stepwise_run(i)
    end_time = time.time()  # palok
    elapsed_time = end_time - start_time  # calculate the difference
    # f.calculate_toa_ocean_airborne_fraction()
    # f.prepare_output_xarrays()
    print(f"One iteration {i} of the loop took {elapsed_time} seconds to run.")

One iteration 0 of the loop took 0.015509843826293945 seconds to run.
One iteration 1 of the loop took 0.012490034103393555 seconds to run.
One iteration 2 of the loop took 0.010169267654418945 seconds to run.
One iteration 3 of the loop took 0.009487152099609375 seconds to run.
One iteration 4 of the loop took 0.009992361068725586 seconds to run.
One iteration 5 of the loop took 0.00974583625793457 seconds to run.
One iteration 6 of the loop took 0.010032176971435547 seconds to run.
One iteration 7 of the loop took 0.009733915328979492 seconds to run.
One iteration 8 of the loop took 0.009744882583618164 seconds to run.
One iteration 9 of the loop took 0.009598970413208008 seconds to run.
One iteration 10 of the loop took 0.010422945022583008 seconds to run.
One iteration 11 of the loop took 0.010659933090209961 seconds to run.
One iteration 12 of the loop took 0.011121749877929688 seconds to run.
One iteration 13 of the loop took 0.011138200759887695 seconds to run.
One iteration 14 

In [4]:
temperature = model.get_temperature_array()

temp = temperature[:,0,:,0]